# Set up local SparkSession

In [1]:
from datetime import date

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:0.14.0")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
    .config("spark.sql.catalog.spark_catalog.type", "hive")
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.local.type", "hadoop")
    .config("spark.sql.catalog.local.warehouse", "spark-warehouse")
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.repl.eagerEval.maxNumRows", 10)
    .getOrCreate()
)

22/08/12 16:41:22 WARN Utils: Your hostname, HF-LPT-1115A.local resolves to a loopback address: 127.0.0.1; using 192.168.178.21 instead (on interface en0)
22/08/12 16:41:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/guido.kosloffgancedo/.ivy2/cache
The jars for the packages stored in: /Users/guido.kosloffgancedo/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.2_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a360e4bb-aa4f-4663-99ce-3d13365af1fd;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/guido.kosloffgancedo/Git/pp/jupyter-demo/.venv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;0.14.0 in central
:: resolution report :: resolve 125ms :: artifacts dl 3ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;0.14.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-a360e4bb-aa4f-4663-99ce-3d13365af1fd
	confs: [default]
	0 artifacts copied, 1 already retrieved (0kB/3ms)
22/08/12 16:41:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profil

# Read flights data from CSV

In [3]:
df = (
    spark
    .read
    .option("header", True)
    .option("inferSchema", True)
    .csv("flights.csv")
)
df

year,month,day,departure_time,arrival_time,carrier,flight_number,aircraft_registration,flight_time,airport_origin,airport_destination,distance,is_cancelled,is_diverted
2008,1,3,1343,1451,WN,588,N240WN,68,HOU,LIT,393,0,0
2008,1,3,1125,1247,WN,1343,N523SW,82,HOU,MAF,441,0,0
2008,1,3,2009,2136,WN,3841,N280WN,87,HOU,MAF,441,0,0
2008,1,3,903,1203,WN,3,N308SA,120,HOU,MCO,848,0,0
2008,1,3,1423,1726,WN,25,N462WN,123,HOU,MCO,848,0,0
2008,1,3,2024,2325,WN,51,N483WN,121,HOU,MCO,848,0,0
2008,1,3,1753,2053,WN,940,N493WN,120,HOU,MCO,848,0,0
2008,1,3,622,935,WN,2621,N266WN,133,HOU,MCO,848,0,0
2008,1,3,1944,2210,WN,389,N266WN,146,HOU,MDW,937,0,0
2008,1,3,1453,1716,WN,519,N514SW,143,HOU,MDW,937,0,0


# Perform some simple aggregations and transformations

In [4]:
df = (
    df
    .withColumn("date", F.to_date(F.concat("year", F.lit("-"), "month", F.lit("-"), "day")))
    .where("is_cancelled = 0 and is_diverted = 0")
    .groupBy("date", "airport_origin", "airport_destination")
    .agg({"*": "count", "flight_time": "avg"})
    .withColumnRenamed("count(1)", "count_flights")
    .withColumnRenamed("avg(flight_time)", "avg_flight_time")
    .withColumn("avg_flight_time", F.col("avg_flight_time").cast("int"))
)
df.orderBy("count_flights", ascending=False)

date,airport_origin,airport_destination,count_flights,avg_flight_time
2008-01-03,LAX,OAK,20,80
2008-01-03,LAS,PHX,18,64
2008-01-03,LAX,LAS,14,63
2008-01-03,LAS,LAX,14,67
2008-01-03,LAS,SAN,14,63
2008-01-03,LAS,BUR,13,62
2008-01-03,LAX,PHX,13,66
2008-01-03,LAX,SJC,13,64
2008-01-03,LAS,RNO,13,74
2008-01-03,LAS,OAK,13,95


# Write to partitioned Iceberg table

In [6]:
(
    df
    .writeTo("local.default.flights")
    .partitionedBy("date")
    .createOrReplace()
)

# Perform updates, deletes and merges on the table

### Update and delete

In [7]:
spark.sql("""
    UPDATE local.default.flights
    SET avg_flight_time = NULL
    WHERE avg_flight_time = 80
""")

spark.sql("""
    DELETE FROM local.default.flights
    WHERE airport_origin = 'LAS' or airport_destination = 'LAS'
""")

spark.sql("""
    SELECT *
    FROM local.default.flights
    ORDER BY count_flights DESC
""")

date,airport_origin,airport_destination,count_flights,avg_flight_time
2008-01-03,LAX,OAK,20,null
2008-01-03,LAX,PHX,13,66
2008-01-03,LAX,SJC,13,64
2008-01-03,MCI,MDW,12,81
2008-01-03,MDW,MCI,12,79
2008-01-03,HOU,MSY,11,58
2008-01-03,MHT,BWI,11,79
2008-01-03,LAX,SMF,10,71
2008-01-03,MDW,STL,10,59
2008-01-03,HRL,HOU,9,54


### Merge

In [8]:
incoming_df = spark.createDataFrame([
    {"date": date(2008, 1, 3), "airport_origin": "LAX", "airport_destination": "OAK", "count_flights": 21, "avg_flight_time": 83},
    {"date": date(2008, 1, 4), "airport_origin": "JFK", "airport_destination": "EZE", "count_flights": 22, "avg_flight_time": 660},
])
incoming_df.createOrReplaceTempView("flights_incoming_data")

spark.sql("""
    MERGE INTO local.default.flights t
    USING flights_incoming_data s
    ON t.date = s.date
        AND t.airport_origin = s.airport_origin
        AND t.airport_destination = s.airport_destination
    WHEN MATCHED THEN UPDATE SET t.count_flights = s.count_flights, t.avg_flight_time = s.avg_flight_time
    WHEN NOT MATCHED THEN INSERT *
""")

spark.sql("""
    SELECT *
    FROM local.default.flights
    ORDER BY count_flights DESC
""")

date,airport_origin,airport_destination,count_flights,avg_flight_time
2008-01-04,JFK,EZE,22,660
2008-01-03,LAX,OAK,21,83
2008-01-03,LAX,SJC,13,64
2008-01-03,LAX,PHX,13,66
2008-01-03,MCI,MDW,12,81
2008-01-03,MDW,MCI,12,79
2008-01-03,HOU,MSY,11,58
2008-01-03,MHT,BWI,11,79
2008-01-03,LAX,SMF,10,71
2008-01-03,MDW,STL,10,59


# Time travel

### Query table history and older snapshots

In [9]:
spark.sql("""
    SELECT *
    FROM local.default.flights.history
""")

made_current_at,snapshot_id,parent_id,is_current_ancestor
2022-08-12 16:41:...,5958846659430833687,null,true
2022-08-12 16:41:...,3964990362212599387,5958846659430833687,true
2022-08-12 16:41:...,5520539507180026574,3964990362212599387,true
2022-08-12 16:42:...,1760527321587632917,5520539507180026574,true


In [10]:
spark.sql("""
    SELECT *
    FROM local.default.flights.snapshots
""")

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2022-08-12 16:41:...,4741357521111167692,null,append,spark-warehouse/d...,{spark.app.id -> ...
2022-08-12 16:41:...,5958846659430833687,null,append,spark-warehouse/d...,{spark.app.id -> ...
2022-08-12 16:41:...,3964990362212599387,5958846659430833687,overwrite,spark-warehouse/d...,{spark.app.id -> ...
2022-08-12 16:41:...,5520539507180026574,3964990362212599387,overwrite,spark-warehouse/d...,{spark.app.id -> ...
2022-08-12 16:42:...,1760527321587632917,5520539507180026574,overwrite,spark-warehouse/d...,{spark.app.id -> ...


In [11]:
(
    spark
    .read
    .option("snapshot-id", "5958846659430833687")
    .table("local.default.flights")
    .orderBy("count_flights", ascending=False)
)

date,airport_origin,airport_destination,count_flights,avg_flight_time
2008-01-03,LAX,OAK,20,80
2008-01-03,LAS,PHX,18,64
2008-01-03,LAX,LAS,14,63
2008-01-03,LAS,LAX,14,67
2008-01-03,LAS,SAN,14,63
2008-01-03,LAS,BUR,13,62
2008-01-03,LAX,PHX,13,66
2008-01-03,LAX,SJC,13,64
2008-01-03,LAS,RNO,13,74
2008-01-03,LAS,OAK,13,95


### Rollback to older snapshot

In [12]:
spark.sql("""
    CALL local.system.rollback_to_snapshot(table => 'local.default.flights', snapshot_id => 5958846659430833687)
""")

previous_snapshot_id,current_snapshot_id
1760527321587632917,5958846659430833687


# Remove old snapshots

In [13]:
spark.sql("""
    CALL local.system.expire_snapshots(table => 'local.default.flights', older_than => TIMESTAMP '2023-01-01', retain_last => 1)
""")

deleted_data_files_count,deleted_position_delete_files_count,deleted_equality_delete_files_count,deleted_manifest_files_count,deleted_manifest_lists_count
5,0,0,7,4


In [14]:
spark.sql("""
    SELECT *
    FROM local.default.flights.snapshots
""")

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2022-08-12 16:41:...,5958846659430833687,null,append,spark-warehouse/d...,{spark.app.id -> ...


# Compact small files

In [15]:
spark.sql("""
    CALL local.system.rewrite_data_files(table => 'local.default.flights')
""")

rewritten_data_files_count,added_data_files_count
0,0
